# Making predictions for all geometries on the 2D grid

In [ ]:
from ase.db import connect
from spainn.interface import NacCalculator
from schnetpack.transform import MatScipyNeighborList
import ase
import numpy as np
from tqdm import tqdm

The SpaiNN interface provides a ASE-style calculator that loads the Schnetpack style models and converts the predictions into normal ASE Atom properies. This could then be combined with any ASE type of "dynamics".

### Predictions for the PaiNN model

In [ ]:
calc = NacCalculator(model_file="Painn_model/best_model", neighbor_list=MatScipyNeighborList(cutoff=10.0))
atom = ase.Atoms(symbols="CNHHHH")
atom.calc = calc

In [ ]:
db = connect("grid_configuration.db")
pred_energies = np.zeros(shape=(9191, 3))
pred_nacs     = np.zeros(shape=(9191, 3))

In [ ]:
for idx in tqdm(range(len(db))):
    atom.set_positions(db.get(idx+1).positions)
    # NOTE: forces and nacs have the shape (Natoms, Nstates, xyz) -> here (6, 3, 3)
    props = atom.get_properties(['energy', 'smooth_nacs'])
    pred_energies[idx] = props['energy']
    pred_nacs[idx] = np.sum(np.linalg.norm(props['smooth_nacs'], axis=2), axis=0)

In [ ]:
np.savez("Predictions_Painn.npz", energy = pred_energies, nacs = pred_nacs)

## Predictions for the SchNet model

In [ ]:
calc = NacCalculator(model_file="Schnet_model/best_model", neighbor_list=MatScipyNeighborList(cutoff=10.0))
atom = ase.Atoms(symbols="CNHHHH")
atom.calc = calc

In [ ]:
db = connect("grid_configuration.db")
pred_energies = np.zeros(shape=(9191, 3))
pred_nacs     = np.zeros(shape=(9191, 3))

In [ ]:
for idx in tqdm(range(len(db))):
    atom.set_positions(db.get(idx+1).positions)
    # NOTE: forces and nacs have the shape (Natoms, Nstates, xyz) -> here (6, 3, 3)
    props = atom.get_properties(['energy', 'smooth_nacs'])
    pred_energies[idx] = props['energy']
    pred_nacs[idx] = np.sum(np.linalg.norm(props['smooth_nacs'], axis=2), axis=0)

In [ ]:
np.savez("Predictions_Schnet.npz", energy = pred_energies, nacs = pred_nacs)